# EMI modeling in FEniCS


## Model equations 
In summary, the non-stationary EMI (extra-, membrane, intra-) model for a single cell surrounded by an extracellular domain (as illustrated), for $t\in(0,T]$, is given by 
\begin{align}
    & -\nabla \cdot (\sigma_e\nabla u_e(\mathbf{x},t))=f_e && \quad \mathrm{for} \quad \mathbf{x}\in\Omega_e, \\ 
    & -\nabla \cdot (\sigma_i\nabla u_i(\mathbf{x},t))=f_i && \quad \mathrm{for} \quad \mathbf{x}\in\Omega_i, \label{eq::EMI_2}\\
    & \sigma_e\nabla u_e(\mathbf{x},t)\cdot\mathbf{n}_e = -\sigma_i\nabla u_i(\mathbf{x},t)\cdot\mathbf{n}_i\equiv I_m(\mathbf{x},t) && \quad \mathrm{for} \quad \mathbf{x}\in\Gamma, \label{eq::EMI_3} \\
    & u_i(\mathbf{x},t)-u_e(\mathbf{x},t) = v(\mathbf{x},t) && \quad \mathrm{for} \quad \mathbf{x}\in\Gamma \label{eq::EMI_4} \\ 
    & \frac{\partial v(\mathbf{x},t)}{\partial t} = \frac{1}{C_m}(I_m(\mathbf{x},t) - I_{\text{ion}}(\mathbf{x},t)) && \quad \mathrm{for} \quad \mathbf{x}\in\Gamma,\label{eq::EMI_5}
\end{align}
for the intra- and extra-cellular potentials $u_i,u_e$, and for the membrane current $I_m$:
with $\sigma_e,\sigma_i$ conductivities and $C_m$ the membrane capacitance. The ionic current $I_{\text{ion}}=I_{\text{ion}}(v)$ depends on the specific ionic model of choice. Source terms $f_e,f_i$ are typically zero in the physical setting.


We can close the EMI problem with homogeneous boundary conditions:
\begin{align}
     u_e(\mathbf{x},t) = 0 & \quad \mathrm{for} \quad \mathbf{x}\in\partial\Gamma_e^D,\\ \label{bc2}
     \sigma_e \nabla u_e (\mathbf{x},t) \cdot \mathbf{n}_e = 0 & \quad \mathrm{for} \quad \mathbf{x}\in\partial\Gamma_e^N,
\end{align}
and initial membrane potential $v(\mathbf{x},0)=v_0(\mathbf{x})$.

| <img src="figures/cell.png" width=500> |
|:--:|
|*Modeling Excitable Tissue The EMI Framework, Chapter 5* by M. Kuchta et. al. |

### Exercise 
The EMI solution consist of the intra- and extra- potentials, as well as a membrane term encoded in the current $I_m$. It is possible to eliminate the latter unknown, expressing it as a function of $u_i$ and $u_e$. Try to obtain such an expression for $I_m$; to do so:

1) Discretize in time the membrane equations, i.e. consider $N_t>0$ discrete time steps $0=t_0<t_1<\cdots<t_{N_t-1}=T$, with $\Delta t = t_n - t_{n-1}$ for $n=1,...,N_t$. 

2) Consider a forward finite difference stencil for $\partial/\partial_t$.

3) Consider an implicit treatment of the membrane current $I_m$ and an explicit one of the ionic current $I_{\text{ion}}$.

4) Derive an expression for $I_m$.


### Solution
For the first time step (i.e. $t=t_1$) we have:
$$I_m= \frac{C_m}{\Delta t}(u_i-u_e - v_0)+ I_{\text{ion}}(v_0).$$

## Weak formulation 
After substituting the expression for $I_m$, multiplying by test functions and integrating by parts, the weak form  of the EMI problem reads: given $V_i,V_e$ sufficiently regular Hilbert spaces, with elements satisfying the boundary conditions, find $u_i\in V_i(\Omega_i)$ and $u_e\in V_e(\Omega_e)$ such that

\begin{align}
\sigma_e\int_{\Omega_e} \nabla u_e \cdot\nabla v_e\,\mathrm{d}\mathbf{x} + \frac{C_m}{\Delta t}\int_{\Gamma}u_ev_e\,\mathrm{d}s - \frac{C_m}{\Delta t}\int_{\Gamma}u_iv_e\,\mathrm{d}s & = \int_{\Omega_e} f_e v_e\,\mathrm{d}\mathbf{x} -\int_{\Gamma}fv_e\,\mathrm{d}s,  \\
\sigma_i\int_{\Omega_i}\nabla u_i \cdot\nabla v_i\,\mathrm{d}\mathbf{x} + \frac{C_m}{\Delta t}\int_{\Gamma}u_iv_i\,\mathrm{d}s - \frac{C_m}{\Delta t}\int_{\Gamma}u_ev_i\,\mathrm{d}s & = \int_{\Omega_i} f_i v_e\,\mathrm{d}\mathbf{x} + \int_{\Gamma}fv_i\,\mathrm{d}s, 
\end{align}
for all test functions $v_e\in V_e$ and $v_i\in V_i$.

### Exercise 
Derive an expression for $f$.

### Solution
$$f = \frac{C_m}{\Delta t}v_0 - I_{\text{ion}}(v_0).$$

## Algebraic form
We introduce a yet unspecified discretization via finite element basis functions (e.g. Lagrangian elements of order $p\in\mathbb{N}$ on a regular grid) for $V_e$ and $V_i$:

$$V_{e,h}=\mathrm{span}\left(\{\phi^e_j\}_{j=1}^{N_e}\right),\quad V_{i,h}=\mathrm{span}\left(\{\phi^i_j\}_{j=1}^{N_i}\right),$$

with $N_e,N_i$ and $N_\Gamma\in\mathbb{N}$ denoting the number of degrees of freedom in the corresponding subdomains.

we define the following discrete operators: intra- and extra- Laplacians

\begin{align}
    & A_e=\left[\int_{\Omega_e}\sigma_e\nabla\phi^e_j(\mathbf{x})\cdot\nabla\phi^e_k(\mathbf{x})\,\mathrm{d}\mathbf{x}\right]_{j,k=1}^{N_e}\in\mathbb{R}^{N_e\times N_e}, \\
    & A_i=\left[\int_{\Omega_i}\sigma_i\nabla\phi^i_j(\mathbf{x})\cdot\nabla\phi^i_k(\mathbf{x})\,\mathrm{d}\mathbf{x}\right]_{j,k=1}^{N_i}\in\mathbb{R}^{N_i\times N_i},
\end{align}

membrane mass matrices: 

\begin{align}
    & M_{e}=\left[\int_{\Gamma}\phi^e_j(\mathbf{x})\phi^e_k(\mathbf{x})\,\mathrm{d}s\right]_{j,k=1}^{N_e}\in\mathbb{R}^{N_e\times N_e}, \\
    & M_{i}=\left[\int_{\Gamma}\phi^i_j(\mathbf{x})\phi^i_k(\mathbf{x})\,\mathrm{d}s\right]_{j,k=1}^{N_i}\in\mathbb{R}^{N_i\times N_i},
\end{align}

and the coupling matrix

\begin{align}
    & T_{ei}=\left[\int_{\Gamma}\phi^i_j(\mathbf{x})\phi^e_k(\mathbf{x})\,\mathrm{d}s\right]_{(j,k)=(1,1)}^{(N_e,N_i)}\in\mathbb{R}^{N_e\times N_i}.
    % & T_{ie}=\left[C\int_{\Omega_i}\sigma_i\nabla\phi^i_j(\mathbf{x})\cdot\nabla\phi^i_k(\mathbf{x})\right]_{j,k=1}^{N_i}\in\mathbb{R}^{N_i\times N_i}.
\end{align}

## Exercise
Finally we express the linear system of size $n = N_e+N_i$ corresponding to presented weak formulation. 
Given the latter matrices definitions, write down the $2\times2$ block system corresponding to the presented weak formulation. Can you notice something about the arising overall EMI linear operator, in terms of algebraic properties?

## Solution
\begin{equation}
	\begin{bmatrix}
		CA_e + M_e & T_{ei} \\
		T_{ei}^T & CA_i + M_i \\
	\end{bmatrix}
	\begin{bmatrix}
		\mathbf{u}_e \\
		\mathbf{u}_i \\
	\end{bmatrix} =
	\begin{bmatrix}
		\mathbf{f}_e \\
		\mathbf{f}_i \\
	\end{bmatrix} \, \Longleftrightarrow \, A_n \mathbf{u} = \mathbf{f},
\end{equation}

with $\mathbf{u}_e,\mathbf{f}_e\in\mathbb{R}^{N_e}$ (resp. $\mathbf{u}_i,\mathbf{f}_i\in\mathbb{R}^{N_i}$) the unknowns and the right hand side corresponding to $\Omega_e$ (resp. $\Omega_i$). The EMI operator $A_n$ is symmetric. 


## Test case
To start with, we consider a benchmark example in the 2D unit square where an analytical solution is known:

\begin{align}
& u_i(x,y,t) = sin(2\pi x) \cdot sin(2\pi y)\cdot(1 + e^{-t}),\\
& u_e(x,y,t) = sin(2\pi x) \cdot sin(2\pi y).
\end{align}

and the corresponding source functions:

\begin{align}
& f_i(x,y,t) = 8\pi^2sin(2\pi x) \cdot sin(2\pi y)\cdot(1 + e^{-t}),\\
& f_e(x,y,t) = 8\pi^2sin(2\pi x) \cdot sin(2\pi y),
\end{align}

with $\Omega_i = (0.25, 0.75)\times(0.25, 0.75)$ and $\Omega_i\cup\Omega_e=(0,1)\times(0,1)$, cf. figure.

We impose the ionic current $I_{ion} = v$ and consider the following initial condition on the membrane $\Gamma$:

$$ v_0=sin(2\pi x) \cdot sin(2\pi y).$$

### Exercise 
1) Verify that the given solution satisfies the EMI problem.
2) Which is the value of $I_m$?.


## Implementation
Let us import relevant modules:

In [3]:
from dolfin import *
from multiphenics import *
parameters["ghost_mode"] = "shared_facet" # required by dS


ERROR: Could not find a version that satisfies the requirement multiphenics (from versions: none)
ERROR: No matching distribution found for multiphenics


Set discretization and physical parameters, here we are limited to $n\in\{16,32,64,128,256\}$

In [1]:
# space discretization parameters
n = 16 
p = 1

# time discretization parameters
t          = 0.0
dt         = 0.1 
time_steps = 10  

# physical parameters
C_M     = 1.0
sigma_i = 1.0
sigma_e = 1.0

Set anaytical solution and initial condition (replace TODO with the appropriate expressions):

In [ ]:
# initial membrane potential V
v = Expression("sin(2*pi*x[0]) * sin(2*pi*x[1])", degree = 4)

# source factors
# f_i = Expression("TODO", degree = 4, t = t)
# f_e = Expression("TODO",                   degree = 4)

f_i = Expression("8*pi*pi*sin(2*pi*x[0]) * sin(2*pi*x[1]) * (1.0 + exp(-t))", degree = 4, t = t)
f_e = Expression("8*pi*pi*sin(2*pi*x[0]) * sin(2*pi*x[1])",                   degree = 4)

Import meshes:

In [ ]:
# Mesh
mesh = Mesh("meshes/square/square" + str(n) + ".xml")
subdomains = MeshFunction("size_t", mesh, "meshes/square/square_physical_region" + str(n) + ".xml")
boundaries = MeshFunction("size_t", mesh, "meshes/square/square_facet_region" + str(n) + ".xml")
# Restrictions
omega_i = MeshRestriction(mesh, "meshes/square/square_restriction_om_i" + str(n) + ".rtc.xml")
omega_e = MeshRestriction(mesh, "meshes/square/square_restriction_om_e" + str(n) + ".rtc.xml")

Create FE function spaces and corresponding test/trial functions:

In [ ]:
# Function space
V = FunctionSpace(mesh, "Lagrange", p)
# Block function space
W = BlockFunctionSpace([V, V], restrict=[omega_i, omega_e])

# TRIAL/TEST FUNCTIONS #
uu = BlockTrialFunction(W)
vv = BlockTestFunction(W)

(ui, ue) = block_split(uu)
(vi, ve) = block_split(vv)

# MEASURES #
dx = Measure("dx")(subdomain_data=subdomains)
dS = Measure("dS")(subdomain_data=boundaries)
dS = dS(2) # restrict to the interface, which has facet ID equal to 2


In [5]:
# ASSEMBLE #
a11 = dt * inner(sigma_i*grad(ui), grad(vi))*dx(1) + C_M * inner(ui('-'), vi('-'))*dS
a22 = dt * inner(sigma_e*grad(ue), grad(ve))*dx(2) + C_M * inner(ue('+'), ve('+'))*dS
a12 = - C_M * inner(ue('+'), vi('-'))*dS
a21 = - C_M * inner(ui('-'), ve('+'))*dS

a = [[a11, a12],
    [ a21, a22]]

# boundary conditions
bc_e = DirichletBC(W.sub(1), Constant(0.), boundaries, 1)
bcs  = BlockDirichletBC([None,bc_e])

# ASSEMBLE #
A = block_assemble(a)
bcs.apply(A)

# solution vector
U = BlockFunction(W)

NameError: name 'inner' is not defined

Perform the time loop:

In [ ]:
# Time-stepping
for i in range(time_steps):

    print('Time step', i + 1)  

    # update current time
    t += dt
    
    # update source term 
    f_i.t = t   

    # update rhs    
    fg = v - (dt/C_M) * v
    fi = dt * inner(f_i, vi)*dx(1) + C_M * inner(fg, vi('-'))*dS
    fe = dt * inner(f_e, ve)*dx(2) - C_M * inner(fg, ve('+'))*dS
    f =  [fi, fe]
    F = block_assemble(f)
    bcs.apply(F)

    # SOLVE
    block_solve(A, U.block_vector(), F)
    
    # update membrane potential        
    v = U[0] - U[1]

Error analysis (replace TODO):

In [ ]:
# ERROR
ui_exact = Expression("(1 + exp(-t)) * sin(2*pi*x[0]) * sin(2*pi*x[1])", degree = 4, t = t)
# ue_exact = Expression("TODO", degree = 4)

ue_exact = Expression("sin(2*pi*x[0]) * sin(2*pi*x[1])", degree = 4)

err_i = inner(U[0] - ui_exact, U[0] - ui_exact)*dx(1)
err_e = inner(U[1] - ue_exact, U[1] - ue_exact)*dx(2)
L2_norm_i = sqrt(assemble(err_i))
L2_norm_e = sqrt(assemble(err_e))

# print info
print("~~~~~~~~~~~~~~ Info ~~~~~~~~~~~~~~")
print("dt =", dt)
print("n =", n)
print("p =", p)

print("~~~~~~~~~~~~~~ Errors ~~~~~~~~~~~~~~")
print('L2 error interior:', L2_norm_i)
print('L2 error exterior:', L2_norm_e)

### Exercise
Check the convergence order for various $p=1,2,3,...$ varying $n$. 

## Astrocyte example

In [ ]:
from dolfin       import *
from multiphenics import *
parameters["ghost_mode"] = "shared_facet" # required by dS

# MESHES #
# Mesh
print('REading meshes...')
mesh = Mesh(MPI.comm_world)
mesh = Mesh("meshes/astrocyte/astrocyte_mesh.xml")
subdomains = MeshFunction("size_t", mesh, "meshes/astrocyte/astrocyte_mesh_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "meshes/astrocyte/astrocyte_mesh_facet_region.xml")
mesh.coordinates()[:] *= 1e-6
# Restrictions
interior  = MeshRestriction(mesh, "meshes/astrocyte/astrocyte_mesh_interior_restriction.rtc.xml")
exterior  = MeshRestriction(mesh, "meshes/astrocyte/astrocyte_mesh_exterior_restriction.rtc.xml")
interface = MeshRestriction(mesh, "meshes/astrocyte/astrocyte_mesh_interface_restriction.rtc.xml")

# print size
print('#Mesh cells =', mesh.num_cells())

# forcing term on interface (you can play with this)
f_gamma = Expression('10*sin(x[0]+x[1])', degree=1)

# FUNCTION SPACES #
# Function spaces
V = FunctionSpace(mesh, "Lagrange", 2)

# Block function space
W = BlockFunctionSpace([V, V], restrict=[interior, exterior])

# TRIAL/TEST FUNCTIONS #
uu = BlockTrialFunction(W)
vv = BlockTestFunction(W)

(ui, ue) = block_split(uu)
(vi, ve) = block_split(vv)

# MEASURES #
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)
dS = dS(5) # restrict to the interface, which has facet ID equal to 5

# ASSEMBLE #
print('Assembling...')
# Interior mesh has subdomain id = 3
# Exterior mesh has subdomain id = 1
a11 = inner(grad(ui), grad(vi))*dx(3) + inner(ui('-'), vi('-'))*dS
a22 = inner(grad(ue), grad(ve))*dx(1) + inner(ue('+'), ve('+'))*dS
a12 = - inner(ue('-'), vi('+'))*dS
a21 = - inner(ui('+'), ve('-'))*dS

a = [[a11, a12],
    [a21, a22]]

f1 = -inner(f_gamma, vi('-'))*dS
f2 =  inner(f_gamma, ve('+'))*dS

f =  [f1, f2]

# Outer shell of exterior mesh has facet tag 4
bce = DirichletBC(W.sub(1), Constant(0.), boundaries, 4)
bcs = BlockDirichletBC([None, bce])

A = block_assemble(a)
F = block_assemble(f)
bcs.apply(A)
bcs.apply(F)
U = BlockFunction(W)

print('Solving...')
block_solve(A, U.block_vector(), F, linear_solver = 'mumps')

# save output
print('Writing output...')
U[0].rename('u_i', '')
U[1].rename('u_e', '')
out_i = XDMFFile(MPI.comm_world, "astrocyte_sol_i.xdmf")
out_e = XDMFFile(MPI.comm_world, "astrocyte_sol_e.xdmf")  
out_i.write(U[0])
out_e.write(U[1])
out_i.write(subdomains)
out_e.write(subdomains)

### Visualization
Open the output with Paraview.

1) Use the 'Threshold' filter to isolate extra- (f = 1) and intra- (f = 3) solutions.
2) Intra- and extra- sapces are labelled with scalar 'f' with tags 3 amd 1 respectively.
3) Apply the 'Clip' filter on the extra- data, to visualize both solutions togheter.
4) Rescale and play with colorbars and opacities.


### Figure

| <img src="figures/astrocyte2.png" width=500> |
|:--:|